In [ ]:
"""Tutorial: Describing the implementation of density-fitted MP2 from an RHF reference"""

__author__    = "Dominic A. Sirianni"
__credit__    = ["Dominic A. Sirianni", "Daniel G. A. Smith"]

__copyright__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2017-05-24"

# Density Fitted MP2

As we saw in tutorial (5a), the single most expensive step for a conventional MP2 program using full ERIs is the integral transformation from the atomic orbital (AO) to molecular orbital (MO) basis, scaling as ${\cal O}(N^5)$.  The scaling of this step may be reduced to ${\cal O}(N^4)$ if we employ density fitting, as the three-index density fitted tensors may be transformed individually into the MO basis before being recombined to form the full four-index tensors in the MO basis needed by the MP2 energy expression.  This tutorial will discuss the specific challenges encountered when applying density fitting to an MP2 program.

### Implementation
The first part of our DF-MP2 program will look exactly the same as the conventional MP2 program that we wrote in (5a), with the exception that we must specify the `scf_type df` and omit the option `mp2_type conv` within the `psi4.set_options()` block, to ensure that we are employing density fitting in the Hartree-Fock reference.  Below, implement the following:

- Import Psi4 and NumPy, and set memory & output file
- Define our molecule and Psi4 options
- Compute the RHF reference wavefucntion and energy
- Obtain the number of occupied and virtual MOs, and total number of MOs
- Get the orbital energies and coefficient matrix; partition into occupied & virtual blocks

In [ ]:
# ==> Import statements & Global Options <==
import psi4
import numpy as np

psi4.set_memory(int(2e9))
numpy_memory = 2
psi4.core.set_output_file('output.dat', False)

In [ ]:
# ==> Options Definitions & SCF E, Wfn <==
mol = psi4.geometry("""
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")


psi4.set_options({'basis': 'aug-cc-pvdz',
                  'scf_type': 'df',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

# Get the SCF wavefunction & energies
scf_e, scf_wfn = psi4.energy('scf', return_wfn=True)

# Number of Occupied orbitals & MOs
ndocc = scf_wfn.nalpha()
nmo = scf_wfn.nmo()
nvirt = nmo - ndocc

# Get orbital energies, cast into NumPy array, and separate occupied & virtual
eps = np.asarray(scf_wfn.epsilon_a())
e_ij = eps[:ndocc]
e_ab = eps[ndocc:]

# Get MO coefficients from SCF wavefunction
C = np.asarray(scf_wfn.Ca())
Cocc = C[:, :ndocc]
Cvirt = C[:, ndocc:]

From the conventional MP2 program, we know that the next step is to obtain the ERIs and transform them into the MO basis using the orbital coefficient matrix, **C**.  In order to do this using density-fitted integrals, must first build and transform the DF-ERI's similar to that in the density-fitted HF chapter.

In [ ]:
# ==> Density Fitted ERIs <==
# Build auxiliar basis set
aux = psi4.core.BasisSet.build(mol, "DF_BASIS_SCF", "", "JKFIT", "aug-cc-pVDZ")

# Build instance of Mints object
orb = scf_wfn.basisset()
mints = psi4.core.MintsHelper(orb)

# Build a zero basis
zero_bas = psi4.core.BasisSet.zero_ao_basis_set()

# Raw 3-index
Ppq = np.squeeze(mints.ao_eri(zero_bas, aux, orb, orb))

# Build and invert the Coulomb metric
metric = mints.ao_eri(zero_bas, aux, zero_bas, aux)
metric.power(-0.5, 1.e-14)
metric = np.squeeze(metric)

Qpq = np.einsum("QP,Ppq->Qpq", metric, Ppq)

Now that we have our three-index integrals, we are able to transform them into the MO basis.  To do this, we could simply use `np.einsum()` to carry out the transformation in a single step:
~~~python
# Transform Qpq -> Qmo @ O(N^5)
Qmo = np.einsum('pi,Qpq,qj->Qij', C, Qpq, C)
~~~
This simple transformation works, but it doesn't reduce the caling of the transformation.  This approach saves over the conventional one only because a single ${\cal O}(N^5)$ transformation would need to be done, instead of four.  We can, however, borrow the idea from conventional MP2 to carry out the transformation in more than one step, saving the intermediates along the way.  Using this approach, we are able to transform the `Qpq` tensors into the MO basis in two successive ${\cal O}(N^4)$ steps.  In the cell below, transform the `Qpq` tensors with this reduced scaling algorithm, and save the occupied-virtual slice of the full `Qmo` tensor:

In [ ]:
# ==> Transform Qpq -> Qmo @ O(N^4) <==
Qmo = np.einsum('pi,Qpq->Qiq', C, Qpq)
Qmo = np.einsum('Qiq,qj->Qij', Qmo, C)

# Get Occupied-Virtual Block
Qmo = Qmo[:, :ndocc, ndocc:]

We are now ready to compute the DF-MP2 correlation energy $E_0^{(2)}$.  One approach for doing this would clearly be to form the four-index OVOV $(ia\mid jb)$ ERI tensor directly [an ${\cal O}(N^5)$ contraction], and proceed exactly as we did for conventional MP2.  This would, however, result in needing to store this entire tensor in memory, which would be prohibitive for large systems/basis sets and would only result in minimal savings.  A more clever (and much less memory-intensive) algorithm can be found by considering the MP2 correlation energy expressions,

\begin{equation}
E_{\rm 0,\,SS}^{(2)} = \sum_{ij}\sum_{ab}\frac{(ia\mid jb)[(ia\mid jb) - (ib\mid ja)]}{\epsilon_i - \epsilon_a + \epsilon_j - \epsilon_b},\,{\rm and}
\end{equation}
\begin{equation}
E_{\rm 0,\,OS}^{(2)} = \sum_{ij}\sum_{ab}\frac{(ia\mid jb)(ia\mid jb)}{\epsilon_i - \epsilon_a + \epsilon_j - \epsilon_b},
\end{equation}

for particular values of the occupied orbital indices $i$ and $j$:

\begin{equation}
E_{\rm 0,\,SS}^{(2)}(i, j) = \sum_{ab}\frac{I_{ab}[I_{ab} - I_{ba}]}{\epsilon_i + \epsilon_j - \boldsymbol{\epsilon}_{ab}}
\end{equation}
\begin{equation}
E_{\rm 0,\,OS}^{(2)}(i, j) = \sum_{ab}\frac{I_{ab}I_{ab}}{\epsilon_i + \epsilon_j - \boldsymbol{\epsilon}_{ab}},
\end{equation}

for virtual-virtual blocks of the full ERI tensors $I_{ab}$ and a matrix $\boldsymbol{\epsilon}_{ab}$ containing all possible combinations of the virtual orbital energies $\epsilon_a$ and $\epsilon_b$.  These expressions are advantageous because they only call for two-index contractions between the virtual-virtual blocks of the OVOV ERI tensor, and the storage of only the VV-block of this tensor in memory.  Furthermore, the formation of the $I_{ab}$ tensor is also ameliorated, since only the auxiliary-virtual blocks of the three-index `Qmo` tensor must be contracted, which can be done on-the-fly as opposed to beforehand (requiring no storage in memory).  In practice, these expressions can be used within explicit loops over occupied indices $i$ and $j$; therefore the overall scaling of this step is still ${\cal O}(N^5)$ (formation of $I_{ab}$ is ${\cal O}(N^3)$ inside two loops), however the the drastically reduced memory requirements result in this method a significant win over conventional MP2.

One potentially mysterious quantity in the frozen-index expressions given above is the virtual-virtual orbital eigenvalue tensor, **$\epsilon$**.  To build this array, we can again borrow an idea from our implementation of conventional MP2: reshaping and broadcasting.  In the cell below, use these techniques to build the VV $\boldsymbol{\epsilon}_{ab}$ tensor.

Hint: In the frozen-index expressions above, $\boldsymbol{\epsilon}_{ab}$ is *subtracted* from the occupied orbital energies $\epsilon_i$ and $\epsilon_j$.  Therefore, the virtual orbital energies should be added together to have the correct sign!

In [ ]:
# ==> Build VV Epsilon Tensor <==
e_vv = e_ab.reshape(-1, 1) + e_ab

In addition to the memory savings incurred by generating VV-blocks of our ERI tensors on-the-fly, we can exploit the permutational symmetry of these tensors [Sherrill:ERI] to drastically reduce the number of loops (and therefore Qv,Qv contractions!) which are needed to compute the MP2 correlation energy.  To see the relevant symmetry, recall that a spin-free four index ERI over spatial orbitals (written in chemists' notation) is given by

$$(i\,a\mid j\,b) = \int{\rm d}^3{\bf r}_1{\rm d}^3{\bf r}_2\phi_i^*({\bf x}_1)\phi_a({\bf x}_1)\frac{1}{r_{12}}\phi_j^*({\bf x}_2)\phi_b({\bf x}_2)$$

For real orbitals, it is easy to see that $(i\,a\mid j\,b) = (j\,b\mid i\,a)$; therefore, it is unnecessary to iterate over all combinations of $i$ and $j$, since the value of the contractions containing either $(i\,a\mid j\,b)$ or $(j\,b\mid i\,a)$ will be identical.  Therefore, it suffices to iterate over all $i$ and only $j\geq i$.  Then, the "diagonal elements" ($i = j$) will contribute once to each of the same-spin and opposite-spin correlation energies, and the "off-diagonal" elements ($i\neq j$) will contribute twice to each correlation energy due to symmetry.  This corresponds to placing either a 1 or a 2 in the numerator of the energy denominator, i.e., 

\begin{equation}
E_{denom} = \frac{\alpha}{\epsilon_i + \epsilon_j - \boldsymbol{\epsilon}_{ab}};\;\;\;\alpha = \begin{cases}1;\; i=j\\2;\;i\neq j\end{cases},
\end{equation}

before contracting this tensor with $I_{ab}$ and $I_{ba}$ to compute the correlation energy.  In the cell below, compute the same-spin and opposite-spin DF-MP2 correlation energies using the frozen-index expressions 3 and 4 above, exploiting the permutational symmetry of the full $(ia\mid jb)$ ERIs.  Then, using the correlation energies, compute the total MP2 energy using the DF-RHF energy we computed above.

In [ ]:
mp2_os_corr = 0.0
mp2_ss_corr = 0.0
for i in range(ndocc):
    # Get epsilon_i from e_ij
    e_i = e_ij[i]
    
    # Get 2d array Qa for i from Qov
    i_Qa = Qmo[:, i, :]
    
    for j in range(i, ndocc):
        # Get epsilon_j from e_ij
        e_j = e_ij[j]
        
        # Get 2d array Qb for j from Qov
        j_Qb = Qmo[:, j, :]
        
        # Compute 2d ERI array for fixed i,j from Qa & Qb
        ij_Iab = np.einsum('Qa,Qb->ab', i_Qa, j_Qb)

        # Compute energy denominator
        if i == j:
            e_denom = 1.0 / (e_i + e_j - e_vv)
        else:
            e_denom = 2.0 / (e_i + e_j - e_vv)

        # Compute SS & OS MP2 Correlation
        mp2_os_corr += np.einsum('ab,ab,ab->', ij_Iab, ij_Iab, e_denom)
        mp2_ss_corr += np.einsum('ab,ab,ab->', ij_Iab, ij_Iab - ij_Iab.T, e_denom)

# Compute MP2 correlation & total MP2 Energy
mp2_corr = mp2_os_corr + mp2_ss_corr
MP2_E = scf_e + mp2_corr

In [ ]:
# ==> Compare to Psi4 <==
psi4.compare_values(psi4.energy('mp2'), MP2_E, 6, 'MP2 Energy')

## References

1. Original paper: "Note on an Approximation Treatment for Many-Electron Systems"
	> [[Moller:1934:618](https://journals.aps.org/pr/abstract/10.1103/PhysRev.46.618)] C. Møller and M. S. Plesset, *Phys. Rev.* **46**, 618 (1934)
2. The Laplace-transformation in MP theory: "Minimax approximation for the decomposition of energy denominators in Laplace-transformed Møller–Plesset perturbation theories"
    > [[Takasuka:2008:044112](http://aip.scitation.org/doi/10.1063/1.2958921)] A. Takatsuka, T. Siichiro, and W. Hackbusch, *J. Phys. Chem.*, **129**, 044112 (2008)
3. Equations taken from:
	> [[Szabo:1996](https://books.google.com/books?id=KQ3DAgAAQBAJ&printsec=frontcover&dq=szabo+%26+ostlund&hl=en&sa=X&ved=0ahUKEwiYhv6A8YjUAhXLSCYKHdH5AJ4Q6AEIJjAA#v=onepage&q=szabo%20%26%20ostlund&f=false)] A. Szabo and N. S. Ostlund, *Modern Quantum Chemistry: Introduction to Advanced Electronic Structure Theory*. Courier Corporation, 1996.
4. Algorithms taken from:
	> [Crawford:prog] T. D. Crawford, "The Second-Order Møller–Plesset Perturbation Theory (MP2) Energy."  Accessed via the web at http://github.com/CrawfordGroup/ProgrammingProjects.
5. ERI Permutational Symmetries
	> [Sherrill:ERI] C. David Sherrill, "Permutational Symmetries of One- and Two-Electron Integrals." Accessed via the web at http://vergil.chemistry.gatech.edu/notes/permsymm/permsymm.pdf.